In [1]:
import cv2
import os
import pandas as pd

import torch
from torch import nn
from PIL import Image
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms, datasets 

from torchmetrics.classification import BinaryAccuracy
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from torchmetrics import ConfusionMatrix

import copy
import numpy as np
from tqdm.auto import tqdm
from matplotlib import pyplot as plt

from efficientnet_pytorch import EfficientNet
from utils.trainer import *

In [2]:
def compare_models(models, dataloaders, model_names=None, device="cuda"):
    if model_names is None:
        model_names = [f'Model {i + 1}' for i in range(len(models))]

    metrics = {
        'F1 Score': f1_score,
        'Accuracy': accuracy_score,
        'AUC': roc_auc_score
    }

    results = {}
    for metric_name, metric_func in metrics.items():
        results[metric_name] = {}
        for model_name, model, dataloader in zip(model_names, models, dataloaders):
            for images, labels in tqdm(dataloader):
                
                images = images.to(device)
                labels = torch.eye(2)[labels].to(device)
                
                model.eval()
                with torch.no_grad():
                    outputs = model(images).cpu()
                    if metric_name == 'AUC':
                        y_prob = torch.softmax(outputs, dim=1)
                        y_prob_pos = y_prob[:, 1]
                    y_pred = torch.argmax(outputs, dim=1)

                if metric_name == 'AUC':
                    labels = torch.Tensor([torch.argmax(i).item() for i in labels])
                    score = metric_func(labels.cpu(), y_prob_pos)
                else:
                    labels = torch.Tensor([torch.argmax(i).item() for i in labels])
                    
                    score = metric_func(labels, y_pred)
                results[metric_name][model_name] = score

    return results

def print_results(results):
    for metric_name, model_results in results.items():
        print(f'{metric_name}:')
        for model_name, score in model_results.items():
            print(f'  {model_name}: {score:.4f}')
        print()

In [3]:
# A class build to hold all configurations to train the model
class Configuration:
    def __init__(self, basePath, modelName, EPOCHS, LR, batchSize, imgSize):
        self.trainPath = basePath + "train/"
        self.testPath = basePath + "test/"
        
        self.EPOCHS = EPOCHS
        self.lr = LR
        self.bs = batchSize
        self.Is = (imgSize, imgSize)
        self.train_validate_percent = 0.95
        
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.modelName = modelName

config = Configuration(basePath="../datasets/Retinal_OCT/", modelName="efficientnet-b0", EPOCHS=25, LR=0.0001, batchSize=64, imgSize=32)

In [4]:
A_transform = transforms.Compose([
                                        transforms.Resize(size=(128,128)),
                                        transforms.ToTensor(),
])
A32_transform = transforms.Compose([
                                        transforms.Resize(size=(32,32)),
                                        transforms.ToTensor(),
])
B_transform = transforms.Compose([
                                        transforms.Resize(size=(128,128)),
                                        transforms.ToTensor(),
])

In [5]:
A_test = CustomDataset(config.testPath, transform=A_transform)
A32_test = CustomDataset(config.testPath, transform=A32_transform)
B_test = CustomDataset(config.testPath, transform=B_transform)

A_testLoader = DataLoader(A_test, batch_size=config.bs, shuffle=True, num_workers=0)
A32_testLoader = DataLoader(A32_test, batch_size=config.bs, shuffle=True, num_workers=0)
B_testLoader = DataLoader(B_test, batch_size=config.bs, shuffle=True, num_workers=0)

dataloaders = [A_testLoader, A32_testLoader, B_testLoader]

In [6]:
model = EfficientNet.from_pretrained(config.modelName)

# Prints the inital fully connected classification layer
print(model._fc, "\n")

# Creates new sequential fully connected classification layers 
model._fc = nn.Sequential(
    nn.Linear(in_features= model._fc.in_features, out_features= 512),
    nn.ReLU(),
    nn.Linear(in_features= 512, out_features= 128),
    nn.ReLU(),
    nn.Linear(in_features= 128, out_features= 2),
    nn.Sigmoid(),
)


A = copy.deepcopy(model).to(config.device)
A.load_state_dict(torch.load("models/BC_A.pth"))

A32 = copy.deepcopy(model).to(config.device)
A32.load_state_dict(torch.load("models/BC_A_32.pth"))

B = copy.deepcopy(model).to(config.device)
B.load_state_dict(torch.load("models/BC_B.pth"))

models = [A, B, A32]

Loaded pretrained weights for efficientnet-b0
Linear(in_features=1280, out_features=1000, bias=True) 



In [7]:
results = compare_models(models, dataloaders, model_names=["A", "B", "A32"])
print_results(results)

100%|██████████████████████████████████████████████████████████████████████████████████| 15/15 [00:02<00:00,  5.03it/s]

F1 Score:
  A: 1.0000
  B: 0.5532
  A32: 0.0000

Accuracy:
  A: 1.0000
  B: 0.5588
  A32: 0.5882

AUC:
  A: 1.0000
  B: 0.4786
  A32: 0.4615

